# Scrapy + Scraper API

In [39]:
pip install scrapy

Note: you may need to restart the kernel to use updated packages.


In [40]:
import scrapy
from urllib.parse import urlencode
from urllib.parse import urlparse
import json
from datetime import datetime
API_KEY = 'afd4c282f739aadfe2b28f623ddb2b17'

In [41]:
def get_url(url):
   payload = {'api_key': API_KEY, 'url': url, 'autoparse': 'true', 'country_code': 'fr'}
   proxy_url = 'http://api.scraperapi.com/?' + urlencode(payload)
   return proxy_url

def create_google_url(query, site=''):
   google_dict = {'q': query, 'num': 100, }
   if site:
       web = urlparse(site).netloc
       google_dict['as_sitesearch'] = web
       return 'http://www.google.com/search?' + urlencode(google_dict)
   return 'http://www.google.com/search?' + urlencode(google_dict)

class GoogleSpider(scrapy.Spider):
   name = 'google'
   #allowed_domains = ['api.scraperapi.com']
   #custom_settings = {'ROBOTSTXT_OBEY': False, 'LOG_LEVEL': 'INFO', 'CONCURRENT_REQUESTS_PER_DOMAIN': 5, 'RETRY_TIMES': 0}
   allowed_domains = ['www.google.com']
   custom_settings = {'ROBOTSTXT_OBEY': False, 'LOG_LEVEL': 'INFO', 'CONCURRENT_REQUESTS_PER_DOMAIN': 5, 'RETRY_TIMES': 0}

   def start_requests(self):
       queries = ['innovation'] 
       for query in queries:
           url = create_google_url(query)
           yield scrapy.Request(get_url(url), callback=self.parse, meta={'pos': 0})

   def parse(self, response):
       di = json.loads(response.text)
       pos = response.meta['pos']
       dt = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
       for result in di['organic_results']:
           title = result['title']
           snippet = result['snippet']
           link = result['link']
           item = {'title': title, 'snippet': snippet, 'link': link, 'position': pos, 'date': dt}
           pos += 1
           yield item
       next_page = di['pagination']['nextPageUrl']
       if next_page:
           yield print(scrapy.Request(get_url(next_page), callback=self.parse, meta={'pos': pos}).text)

In [42]:
if __name__ == '__main__':
    result = list(GoogleSpider().start_requests())

In [43]:
result

[<GET http://api.scraperapi.com/?api_key=afd4c282f739aadfe2b28f623ddb2b17&url=http%3A%2F%2Fwww.google.com%2Fsearch%3Fq%3Dinnovation%26num%3D100&autoparse=true&country_code=fr>]

In [44]:
link = str(result[0])[5:-1]
link

'http://api.scraperapi.com/?api_key=afd4c282f739aadfe2b28f623ddb2b17&url=http%3A%2F%2Fwww.google.com%2Fsearch%3Fq%3Dinnovation%26num%3D100&autoparse=true&country_code=fr'

In [45]:
import datetime 

#function that allows to add a "limit date" parameter in the link
def link_sub_month(nbmonths,link):
    today = datetime.date.today()
    
    nbdays=nbmonths*31

    days_to_substract = datetime.timedelta(days=nbdays)

    date = today - days_to_substract
    #limit date
    
    jour=str(date.day)
    mois=str(date.month)
    annee=str(date.year)
    
    link = link+"&source=lnt&tbs=cdr%3A1%2Ccd_min%3A"+mois+"%2F"+jour+"%2F"+annee+"%2Ccd_max%3A&tbm="
    return link

In [47]:
link=link_sub_month(2,link)
link

'http://api.scraperapi.com/?api_key=afd4c282f739aadfe2b28f623ddb2b17&url=http%3A%2F%2Fwww.google.com%2Fsearch%3Fq%3Dinnovation%26num%3D100&autoparse=true&country_code=fr&source=lnt&tbs=cdr%3A1%2Ccd_min%3A9%2F4%2F2020%2Ccd_max%3A&tbm=&source=lnt&tbs=cdr%3A1%2Ccd_min%3A11%2F5%2F2020%2Ccd_max%3A&tbm='

In [48]:
# On importe la fonction 'get' (téléchargement) de 'requests' 
# Et la classe 'Selector' (Parsing) de 'scrapy'
from requests import get
from scrapy import Selector
# Lien de la page à scraper
url = link
response = get(url)
source = None # Le code source de la page 
if response.status_code == 200 :
    # Si la requete s'est bien passee
    source = response.text

In [49]:
source

'{"search_information":{"total_results":1790000000,"time_taken_displayed":0.86,"query_displayed":"innovation"},"ads":[],"knowledge_graph":{"title":"","description":"","source":{"name":""},"related":[],"related_link":null,"social_media":[],"see_more_about":[{"title":"People also search for","link":"https://www.google.com/search?num=100&gl=FR&q=Innovation&stick=H4sIAAAAAAAAAONgFuLQz9U3KMuwLFCCs7SEs5Ot9JPzc3Pz86yKM1NSyxMrixexcnnm5eWXJZZk5uftYGUEANz69kA7AAAA&sa=X&ved=2ahUKEwif6YPO_IbuAhUSGVkFHfwtDQ8QMSgAMHd6BAgmEAE"},{"title":"View 15+ more","link":"https://www.google.com/search?num=100&gl=FR&q=Innovation&stick=H4sIAAAAAAAAAONgFuLQz9U3KMuwLFCCs7SEs5Ot9JPzc3Pz86yKM1NSyxMrixexcnnm5eWXJZZk5uftYGUEANz69kA7AAAA&sa=X&ved=2ahUKEwif6YPO_IbuAhUSGVkFHfwtDQ8QzTooATB3egQIJhAC"},{"title":"Creativity","link":"https://www.google.com/search?num=100&gl=FR&q=Creativity&stick=H4sIAAAAAAAAAONgFuLQz9U3KMuwLFDiBLEMjQzS8rT4nPNzc_PzgjNTUssTK4sXsXI5F6UmlmSWZZZU7mBlBAAGVG_qNwAAAA&sa=X&ved=2ahUKEwif6YPO_IbuAhUSGVkFH

In [50]:
import re
s = source
pattern = re.compile("\"link\"[^,]+")
my_links = list(set([x for x in pattern.findall(s)]))
my_links
my_links = [i[8:] for i in my_links]
my_links = [i[:-1] for i in my_links]

mylinks=[]

for i in my_links:
    if "https://www.google" not in i:
        mylinks.append(i)

mylinks

['https://www.cnrtl.fr/lexicographie/innovation',
 'https://www.enseignementsup-recherche.gouv.fr/pid24627/innovation.html',
 'https://reliefweb.int/job/3701865/senior-officer-advocacy-innovation',
 'https://theinnovationcompany.fr/',
 'http://evene.lefigaro.fr/citations/mot.php?mot=innovation',
 'https://www.mckinsey.com/business-functions/strategy-and-corporate-finance/our-insights/the-eight-essentials-of-innovation',
 'https://finance-innovation.org/',
 'https://www.industrie-techno.com/innovation',
 'https://www.cairn.info/revue-innovations.htm',
 'https://www.retis-innovation.fr/',
 'https://www.education.gouv.fr/journee-de-l-innovation-5198',
 'https://www.airbus.com/innovation.html',
 'https://www.merckgroup.com/en/research/open-innovation/innovation-cup.html',
 'https://www.sival-innovation.com/',
 'https://www.unicef.org/innovation/',
 'https://www.entreprises.gouv.fr/fr/entrepreneuriat/aides-et-financement/credit-d-impot-innovation',
 'https://www.agrisudouest.com/',
 'https: